In [1]:
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install ast
%pip install logging
%pip install seaborn


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached AST-0.0.2.tar.gz (19 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/d8/yhrnl4gn78vbcvy365vfz_q00000gp/T/pip-install-2w6panty/ast_cb7a649db9a44b2ca43cfe93a0d08c4a/setup.py", line 6, in <module>
          README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "<frozen codecs>", line 918, in open
      FileNotFoundError: [Errno 2] No such file or di

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from scipy import stats
import numpy as np
from scipy.stats import pearsonr, spearmanr
import statsmodels.api as sm

In [16]:
# # Load the dataset
# df = pd.read_csv('../CMBabble_Master_clean.csv')


# df['Sex'] = df['Sex'].map({'M': 0, 'F': 1})
# df['Treatment'] = df['Treatment'].map({'CONTROL': 0, 'CORT': 1, 'OIL': 2})
# df['Hatch date'] = pd.to_datetime(df['Hatch date'], errors='coerce')
# df['Babbles_len'] = df['Babbles'].apply(lambda x: len(eval(x)))

# # Basic descriptive statistics
# print("Descriptive Statistics:")
# print(df[['Babbles_len', 'Treatment', 'No. eggs hatched from nest', 'No. birds fledged from nest', 'Age on bout day', 'Days prior to fledging', 'Hatch date', 'Fledge date','Fledge age', 'Date on vocalization', 'Date on vocalization 2']].describe())


# # Histograms for numerical data
# columns = ['Treatment', 'Sex', 'Babbles_len', 'No. eggs hatched from nest', 'Age on bout day']
# axes = df[columns].hist(bins=20, figsize=(15, 12))
# plt.suptitle('Distribution of Babbling Features')

# # Set dynamic titles and axis labels
# for ax, col in zip(axes.flatten(), columns):
#     ax.set_title(f'Distribution of {col}')  # Set dynamic title
#     ax.set_xlabel(col)                     # Set x-axis label
#     ax.set_ylabel('Babbles Count')         # Set y-axis label

# # Show the plot
# plt.show()

# # Box plots to check for outliers
# sns.boxplot(x='Sex', y='Babbles_len', data=df)
# plt.title('Babbling Duration by Sex')
# plt.show()

# # Correlation heatmap
# corr = df[['Treatment', 'Sex', 'Babbles_len']].corr()
# sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
# plt.title('Correlation Matrix')
# plt.show()

In [18]:
# Load the CSV file
data = pd.read_csv('../CMBabble_Master_clean.csv')

# Convert date columns to datetime
date_columns = ['Hatch date', 'Fledge date', 'Date on vocalization']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')

# Extract statistics from 'Babbles' column
def process_babbles(babbles):
    try:
        babble_list = ast.literal_eval(babbles)  # Convert string to list
        return {
            'babble_count': len(babble_list),
            'babble_mean': sum(babble_list) / len(babble_list) if babble_list else 0,
            'babble_sum': sum(babble_list),
        }
    except (ValueError, SyntaxError):
        return {'babble_count': 0, 'babble_mean': 0, 'babble_sum': 0}

babbles_stats = data['Babbles'].apply(process_babbles)
data['Babble Count'] = babbles_stats.apply(lambda x: x['babble_count'])
data['Babble Mean'] = babbles_stats.apply(lambda x: x['babble_mean'])
data['Babble Sum'] = babbles_stats.apply(lambda x: x['babble_sum'])

### Basic exploratory data analysis

In [ ]:
# Scatter plot: Age on bout day vs. Babble Sum
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Age on bout day', y='Babble Sum', hue='Sex', style='Treatment', data=data)
plt.title("Age on Bout Day vs. Total Babble Sum")
plt.xlabel("Age on Bout Day")
plt.ylabel("Total Babble Sum")
plt.legend(title="Legend", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Aggregate data by Nest ID
nest_aggregated = data.groupby('Nest ID').agg({
    'Babble Sum': 'sum',
    'Babble Count': 'sum',
    'Age on bout day': 'mean',  # Example of averaging age for the nest
    'Fledge age': 'mean',       # Average fledge age per nest
    'Sex': lambda x: x.mode()[0] if not x.empty else None,  # Mode for categorical data
}).reset_index()

# Plot total babbles per nest
plt.figure(figsize=(10, 6))
sns.barplot(x='Nest ID', y='Babble Sum', data=nest_aggregated, palette="viridis")
plt.title("Total Babble Sum per Nest")
plt.xlabel("Nest ID")
plt.ylabel("Total Babble Sum")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Grouping by Treatment and Nest ID
grouped_data = data.groupby(['Treatment', 'Nest ID']).agg({
    'Babble Sum': 'sum',
    'Babble Count': 'mean',
    'Age on bout day': 'mean',
    'Days prior to fledging': 'mean',
}).reset_index()

# Bar Plot: Total Babble Sum by Treatment
plt.figure(figsize=(10, 6))
sns.barplot(x='Treatment', y='Babble Sum', data=grouped_data, palette="viridis")
plt.title("Total Babble Sum by Treatment")
plt.xlabel("Treatment")
plt.ylabel("Total Babble Sum")
plt.tight_layout()
plt.show()



### Hypothesis
-   **Null Hypothesis (H₀):** The number of eggs hatched in a nest does not influence the total babbles in the nest.
-   **Alternative Hypothesis (H₁):** The number of eggs hatched in a nest positively influences the total babbles in the nest.

In [ ]:
# Aggregate data by Nest ID
nest_data = data.groupby('Nest ID').agg({
    'Babble Sum': 'sum',  # Total babble sum for the nest
    'Babble Count': 'sum',  # Total babble events for the nest
    'No. eggs hatched from nest': 'first',  # Constant within each nest
    'No. birds fledged from nest': 'first'  # Constant within each nest
}).reset_index()


# Visualize the Relationship

# Scatter Plot: Plot No. eggs hatched from nest vs. Babble Sum.
plt.figure(figsize=(8, 6))
sns.scatterplot(x='No. eggs hatched from nest', y='Babble Sum', hue='Nest ID', data=nest_data, palette='tab20', s=100)
plt.title("Effect of Number of Eggs Hatched on Total Babble Activity by Nest ID")
plt.xlabel("Number of Eggs Hatched from Nest")
plt.ylabel("Total Babble Sum")
plt.legend(title="Nest ID", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# Boxplot/Barplot: Group Babble Sum by categories of No. eggs hatched from nest.
plt.figure(figsize=(8, 6))
sns.boxplot(x='No. eggs hatched from nest', y='Babble Sum', data=nest_data, palette="coolwarm")
plt.title("Babble Sum Distribution by Number of Eggs Hatched")
plt.xlabel("Number of Eggs Hatched from Nest")
plt.ylabel("Total Babble Sum")
plt.show()

# Perform Statistical Testing
# Use Pearson and/or Spearman correlation to check the relationship between No. eggs hatched from nest and Babble Sum
pearson_corr, pearson_p_value = pearsonr(nest_data['No. eggs hatched from nest'], nest_data['Babble Sum'])
spearman_corr, spearman_p_value = spearmanr(nest_data['No. eggs hatched from nest'], nest_data['Babble Sum'])

print(f"Pearson Correlation: {pearson_corr}, P-value: {pearson_p_value}")
print(f"Spearman Correlation: {spearman_corr}, P-value: {spearman_p_value}")

# Fit a regression model to quantify the relationship
X = nest_data['No. eggs hatched from nest']
y = nest_data['Babble Sum']
X = sm.add_constant(X)  # Add intercept to the regression model
model = sm.OLS(y, X).fit()
print(model.summary())

# Control for Potential Confounding Variables
X_full = nest_data[['No. eggs hatched from nest', 'No. birds fledged from nest']]
X_full = sm.add_constant(X_full)
model_full = sm.OLS(y, X_full).fit()
print(model_full.summary())


### Conclusion

-   **R-squared = 0.001**: This means that **No. eggs hatched from nest** explains 0.1% of the variability in **Babble Sum**. It's a decent amount of explanation, but there is still 99.9% of variability that might be explained by other factors (like treatment, sex, etc.).

-   **Coefficient for No. eggs hatched from nest = -242.5290**: This indicates that for each additional egg hatched in the nest, the **Babble Sum** increases by **--242.5290  units**.

-   **P-value for No. eggs hatched from nest =  0.936**: The p-value is **very high** (greater than 0.05), meaning you cannot reject the null hypothesis and conclude that the number of eggs hatched in a nest **does not significantly influence the babble activity**.